In [3]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility --> random 값을 매번 같은 것을 반환함
#sklearn 추가
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [6]:
#여기는 강의안 따라 만들어 본 것
tf.set_random_seed(777)  # for reproducibility --> random 값을 매번 같은 것을 반환함
xy = np.loadtxt('heart.csv',skiprows=1,delimiter=',',dtype=np.float32)

#input data를 normalize 하지 않으면 학습이 제대로 안된다!

# 이런식으로 함수를 만들어도 되고 sklearn을 통해서 normalize 함수를 불러와도 되고
# def MinMaxScaler(data):
#     numerator = data - np.min(data, 0)
#     denominator = np.max(data, 0) - np.min(data, 0)
#     # noise term prevents the zero division
#     return numerator / (denominator + 1e-5)
# xy = MinMaxScaler(xy)
scaler = MinMaxScaler(feature_range=(0,1))
xy= scaler.fit_transform(xy)

x_data = xy[:,0:-1]
y_data = xy[:,[-1]]


x_data_train = xy[:250,0:-1]
x_data_test = xy[250:,0:-1]
y_data_train = xy[:250,[-1]]
y_data_test = xy[250:,[-1]]


print(x_data)

# print(xy)
print(xy.shape)
print("x_data shape:"+str(x_data.shape))
print("y_data shape:"+str(y_data.shape))
# print(x_data_train)
print(x_data_train.shape)
# print(y_data_train)
print(y_data_train.shape)

# print(x_data_test)
print(x_data_test.shape)
# print(y_data_test)
print(y_data_test.shape)

X = tf.placeholder(tf.float32,shape=[None,13])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([13,1]),name='weight')
b = tf.Variable(tf.random_normal([1]),name='bias')

hypothesis = tf.sigmoid(tf.matmul(X,W)+b)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

#predicted 값을 설정하기위해서 hypothesis 값을 어떤식으로 조정해야하지?
#어떻게 해야 좋은게
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], 
                               feed_dict={X: x_data_train, Y: y_data_train})
        if step%100 == 0:
            print(step,cost_val)
    
    h, c, a = sess.run([hypothesis,predicted, accuracy],
                               feed_dict = {X:x_data_test,Y:y_data_test})
    print("\nhypothesis: ",h,"Correct (Y):",c,"Accuracy ",a)


[[0.7083333  1.         1.         ... 0.         0.         0.33333334]
 [0.16666669 1.         0.6666667  ... 0.         0.         0.6666667 ]
 [0.25       0.         0.33333334 ... 1.         0.         0.6666667 ]
 ...
 [0.81250006 1.         0.         ... 0.5        0.5        1.        ]
 [0.5833333  1.         0.         ... 0.5        0.25       1.        ]
 [0.5833333  0.         0.33333334 ... 0.5        0.25       0.6666667 ]]
(303, 14)
x_data shape:(303, 13)
y_data shape:(303, 1)
(250, 13)
(250, 1)
(53, 13)
(53, 1)
0 0.8176665
100 0.78029114
200 0.7548203
300 0.73340213
400 0.7140689
500 0.69623345
600 0.6796521
700 0.6641766
800 0.6496921
900 0.6361012
1000 0.62331927
1100 0.6112716
1200 0.59989274
1300 0.5891248
1400 0.5789168
1500 0.5692238
1600 0.56000584
1700 0.5512275
1800 0.54285717
1900 0.5348668
2000 0.52723116
2100 0.5199274
2200 0.5129353
2300 0.5062362
2400 0.49981323
2500 0.49365115
2600 0.4877358
2700 0.48205426
2800 0.47659466
2900 0.47134587
3000 0.4662977

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import utils


# error fix link 
# :https://www.kaggle.com/pratsiuk/valueerror-unknown-label-type-continuous 
lab_enc = preprocessing.LabelEncoder()

#np.ravel 이게 뭔지 잘 모르겠네

training_scores_y = lab_enc.fit_transform(np.ravel(y_data_train))
print(np.ravel(y_data_train).shape)
lr = LogisticRegression(solver='lbfgs')
lr.fit(x_data_train,training_scores_y)

print("Test Accuracy {:.2f}%".format(lr.score(x_data_test,y_data_test)*100))

(250,)
Test Accuracy 50.94%
